In [23]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [24]:
df = pd.read_csv('./data.csv')
skills = df['entity'].unique()
groups = df.groupby(['id', 'origin']).agg({'entity': set})
groups.head(3)

,,entity
id,origin,
33799,GetOnBoard,{laravel}
33862,GetOnBoard,"{asp.net, javascript, .netcore, c#, css, mvc, ..."
33941,GetOnBoard,"{react, c#, javascript, .net}"


In [27]:
# Create a 2D NumPy array to represent the graph
graph = np.zeros((len(skills), len(skills)))
graphx = defaultdict(set)

count, nodes, edges = 0, 0, 0
for job in groups['entity'].apply(list):
  count += 1
  for i, skill in enumerate(job):
    for other_skill in job[i+1:]:
        edges += 2
        
        graph[list(skills).index(skill), list(skills).index(other_skill)] += 1
        graphx[skill].add(other_skill)

        graph[list(skills).index(other_skill), list(skills).index(skill)] += 1
        graphx[other_skill].add(skill)

print(f'{count} jobs, {len(skills)} skills(nodes) and {edges} edges were analyzed', len(skills)*len(skills))

1041 jobs, 610 skills(nodes) and 14464 edges were analyzed 372100


In [28]:
for j in range(len(skills)):
    sum_columns = graph[:, j].sum()
    if sum_columns != 0:
        graph[:, j] /= sum_columns


In [33]:

# Compute the PageRank score
N = len(skills)
r = np.ones(N) / N  # initial score
for i in range(100):  # number of iterations
    r_new = graph @ r
    if np.allclose(r, r_new, atol=1e-6, rtol=0):
        break
    r = r_new

# Sort skills by score
score_dict = {list(skills)[i]: r[i] for i in range(N)}

def suggest(*skills, limit = 5):
    skills_set = set()
    for s in skills:
        skills_set = skills_set.union(graphx[s])

    sorted_scores = sorted(skills_set - set(skills), key=lambda x: score_dict[x], reverse=True)

    # Print the scores
    return sorted_scores[:limit]

In [34]:
suggest('python', 'sql', 'html')

['javascript', 'java', 'css', 'react', 'nodejs']